In [1]:
import numpy as np
import tensorflow as tf

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
#input data (feature)
#feature(특징1, 특징2)
x_data = np.array(
[ [0,0], [1,0], [1,1], [0,0], [0,0], [0,1] ]
)
print(x_data)

[[0 0]
 [1 0]
 [1 1]
 [0 0]
 [0 0]
 [0 1]]


In [20]:
#one-hot encoding
#classfication
#실측값

#기타=0, 포유류=1, 조류=2
# 기타 = [1, 0, 0]
# 포유류 = [0, 1, 0]
# 조류 = [0, 0, 1]
y_data = np.array([
    [1,0,0],
    [0,1,0],
    [0,0,1],
    [1,0,0],
    [1,0,0],
    [0,1,0]
])

In [4]:
############## 신경망 구성 ##############
# 손실값 구하는 내용은 아직 없음
X = tf.placeholder(tf.float32, name='X')
Y = tf.placeholder(tf.float32, name='Y')

### 가중치와 편향(bias) 설정
### W: [ 입력층(feature 수), 출력층(결과 label 수)]
### 구성이기 때문에 [2,3] 으로 나타낸다.
W = tf.Variable(tf.random_uniform([2,3], -1., 1.))
b = tf.Variable(tf.zeros([3]))

L = tf.add(tf.matmul(X,W), b)
L = tf.nn.relu(L)

신경망 구성은 다음과 비슷하다. <br/>
모든 입력값으로부터 가중치를 곱한 값과 +b 를 하여 활성화 함수를 거치면 출력 값이 나온다. <br/>
<img src="https://deeplearning4j.org/img/multiple_inputs_RBM.png">
출처: https://deeplearning4j.org/kr/restrictedboltzmannmachine

In [5]:
# 출력 값을 사용하기 쉽게 다듬어 주는 과정이라고 한다.
model = tf.nn.softmax(L)

# softmax , 배열내 값들의 합이 1이 되도록 만들어준다.
# 즉, 각각의 확률이라고 볼 수 있다.
# 책에서 예시는, [8.04, 2.76, -6.52] --> [0.52, 0.24, 0.23]

# model은 신경망을 통해 나온 예측값이며,
# 코드에서 예시는, [1,0,0] --> [0.1 0.7 0.2]       [0,1,0] --> [0.2 0.8 0.0]

In [6]:
# cost function
# Cross-Entropy: 원핫 인코딩과 쿵짝이 좋다.

cost = tf.reduce_mean(-tf.reduce_sum(Y * tf.log(model), axis=1))

# 실측값 Y [0,0,1] 에 대하여,
# 예측값 model 이, [0.1 0.6 0.3] 이라면
# tf.log(model)
# [-1.0, -0.2, -0.5]
# Y * tf.log(model)
# [0 , 0 , -0.5]
# 모두 합쳐서 양수로 바꾼 뒤 평균을 냄
#######
# 예측값이 0.6 일때 (vs. 0.3 일때)
# 실측값 1
# log(model): -0.2 (vs. -0.5)
# 로그의 인수가 0~1사이의 값이면, 값이 클 수록 log값(절대값)이 작다.
# 따라서 예측값 확률이 높을 수록 cost가 낮게 측정하는 것이다.˙


In [7]:
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)
train_op = optimizer.minimize(cost)

init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

for step in range(100):
    sess.run(train_op, feed_dict={X: x_data, Y: y_data})
    
    if (step + 1) % 10 == 0:
        print(step + 1, sess.run(cost, feed_dict={X: x_data, Y: y_data}))

10 1.0374199
20 1.0353928
30 1.0334154
40 1.0315528
50 1.0296687
60 1.0278932
70 1.0260978
80 1.0244088
90 1.0226866
100 1.0210732


In [8]:
prediction = tf.argmax(model, axis=1)
target = tf.argmax(Y, axis=1)
print('hypothesis: ', sess.run(prediction, feed_dict={X: x_data}))
print('real value: ', sess.run(target, feed_dict={Y: y_data}))

hypothesis:  [1 2 1 1 1 1]
real value:  [0 1 2 0 0 1]


In [9]:
print(sess.run(model, feed_dict={X: x_data})) #prediction

[[0.33324483 0.33351034 0.33324483]
 [0.1862249  0.32051554 0.49325964]
 [0.14630799 0.46741745 0.38627458]
 [0.33324483 0.33351034 0.33324483]
 [0.33324483 0.33351034 0.33324483]
 [0.27237466 0.47674468 0.25088063]]


In [10]:
is_correct = tf.equal(prediction, target)
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print('correction rate: %.2f' % sess.run(accuracy * 100, feed_dict={X: x_data, Y: y_data}))

correction rate: 16.67


신경망 늘리기

In [11]:
W1 = tf.Variable(tf.random_uniform([2,10], -1., 1.))
W2 = tf.Variable(tf.random_uniform([10,3], -1., 1.))

b1 = tf.Variable(tf.zeros([10]))
b2 = tf.Variable(tf.zeros([3]))

In [12]:
L1 = tf.add(tf.matmul(X, W1), b1)
L1 = tf.nn.relu(L1)

model = tf.add(tf.matmul(L1,W2), b2)

cost 함수

In [13]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=Y, logits=model))

optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
train_op = optimizer.minimize(cost)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.



위 단일 신경망과 동일

In [17]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

for step in range(100):
    sess.run(train_op, feed_dict={X: x_data, Y: y_data})
    
    if (step+1)%10==0:
        print(step+1, sess.run(cost, feed_dict={X: x_data, Y: y_data}))
        
prediction = tf.argmax(model, axis=1)
target = tf.argmax(Y, axis=1)
print('prediction: ', sess.run(prediction, feed_dict={X: x_data}))
print('target(real value): ', sess.run(target, feed_dict={Y: y_data}))

is_correct = tf.equal(prediction, target)
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print('correction rate: %.2f' %sess.run(accuracy * 100, feed_dict={X: x_data, Y: y_data}))

10 0.8206773
20 0.7035122
30 0.58778024
40 0.4585402
50 0.33666432
60 0.23452663
70 0.1576798
80 0.10604661
90 0.07353888
100 0.05358805
prediction:  [0 1 2 0 0 1]
target(real value):  [0 1 2 0 0 1]
correction rate: 100.00


여기서도 확인을 해봅시다.. (책 내용에 없음)

In [19]:
print('X:[[1,1],[0,0],[1,0],[0,1],[1,1]], Y: ', sess.run(prediction, feed_dict={X: [[1,1],[0,0],[1,0],[0,1],[1,1]]}))

X:[[1,1],[0,0],[1,0],[0,1],[1,1]], Y:  [2 0 1 1 2]
